# Model Training Notebook

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

from simulator.objects.policies.architectures import ModelTask
from simulator.objects.policies.architectures.perceptron import MultiLayerPerceptron
from simulator.objects.stock import Stock

torch.manual_seed(0)

In [2]:
def generate_example_stock_features(n_stocks: int) -> torch.Tensor:
    output = []
    noise_dist = torch.distributions.Normal(loc=0, scale=0.001)
    for _ in range(n_stocks):
        cash = torch.rand(size=(1,)) * 110000 - 10000
        earning_value_of_assets = torch.rand(size=(1,)) * 20000 + 10000
        latest_quarterly_earnings = torch.rand(size=(1,)) * 20000 + 10000
        start_price = torch.rand(size=(1,)) * 1000 + 10
        price_slope = torch.rand(size=(1,)) * 0.01 - 0.005
        growth_component = start_price + price_slope * torch.arange(0, 1825)
        noise_component = noise_dist.sample(sample_shape=(1825,))
        price_history = growth_component + noise_component
        quality_of_leadership = torch.rand(size=(1,))
        stock = Stock(
            cash=cash.item(),
            earning_value_of_assets=earning_value_of_assets.item(),
            latest_quarterly_earnings=latest_quarterly_earnings.item(),
            price_history=price_history.numpy(),
            quality_of_leadership=quality_of_leadership.item(),
            stock_volatility=0.5,
        )

        output.append(np.append(stock.get_stock_features(), 0))

    return torch.tensor(output)


class StockDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"device: {device}")

device: cuda


In [4]:
N_SAMPLES = 5000
VALID_RATIO = 0.15
TEST_RATIO = 0.1

# NOTE: SWITCH TO PERCENT ERROR LOSS OR SOME VARIANT

stock_features = generate_example_stock_features(N_SAMPLES)
stock_labels = stock_features[:, 0]

stock_dataset = StockDataset(stock_features, stock_labels)
val_length = int(N_SAMPLES * VALID_RATIO)
test_length = int(N_SAMPLES * TEST_RATIO)
train_length = N_SAMPLES - (val_length + test_length)

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    stock_dataset, lengths=[train_length, val_length, test_length]
)

train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=val_length, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=test_length, shuffle=True)

C:\Users\pomer\AppData\Local\Temp\ipykernel_4576\226733360.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  return torch.tensor(output)
C:\Users\pomer\AppData\Local\Temp\ipykernel_4576\226733360.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
C:\Users\pomer\AppData\Local\Temp\ipykernel_4576\226733360.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.t

In [5]:
N_EPOCHS = 100

model = MultiLayerPerceptron(
    in_channels=14,
    hidden_channels=[16, 32],
    n_classes=1,
    model_task=ModelTask.REGRESSOR,
).to(device)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss_fun = torch.nn.MSELoss()

for i in range(N_EPOCHS):
    train_features, train_labels = next(iter(train_dataloader))
    val_features, val_labels = next(iter(val_dataloader))

    train_features = train_features.to(device)
    train_labels = train_labels.to(device)
    val_features = val_features.to(device)
    val_labels = val_labels.to(device)

    preds = model(train_features)
    loss = loss_fun(preds, train_labels)

    val_preds = model(val_features)
    val_loss = loss_fun(val_preds, val_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch: {i}; training_loss: {loss}; validation_loss: {val_loss}")

Epoch: 0; training_loss: 302519.375; validation_loss: 337940.90625
Epoch: 1; training_loss: 367340.9375; validation_loss: 337773.21875
Epoch: 2; training_loss: 332988.03125; validation_loss: 337579.21875
Epoch: 3; training_loss: 336974.0625; validation_loss: 337354.15625
Epoch: 4; training_loss: 373380.5; validation_loss: 337092.59375
Epoch: 5; training_loss: 327376.28125; validation_loss: 336789.40625
Epoch: 6; training_loss: 328868.34375; validation_loss: 336435.21875
Epoch: 7; training_loss: 334127.375; validation_loss: 336018.5
Epoch: 8; training_loss: 312977.0; validation_loss: 335527.15625
Epoch: 9; training_loss: 303266.40625; validation_loss: 334944.625
Epoch: 10; training_loss: 305365.5625; validation_loss: 334247.65625
Epoch: 11; training_loss: 409364.875; validation_loss: 333409.625
Epoch: 12; training_loss: 336922.09375; validation_loss: 332413.9375
Epoch: 13; training_loss: 342041.15625; validation_loss: 331210.71875
Epoch: 14; training_loss: 325367.3125; validation_loss: 

In [6]:
test_features, test_labels = next(iter(test_dataloader))
print(loss_fun(model(test_features.to(device)), test_labels.to(device)))

tensor(201784.4531, device='cuda:0', grad_fn=<MseLossBackward0>)


In [7]:
torch.save(model, "model_high_prices.pt")